# AMIA 2016 Annual Symposium Workshop (WG13)

## Automated and Scalable Cloud-based RNA-Seq Data Analysis, Part II


Riyue Bao, Ph.D. 
Center for Research Informatics,
The University of Chicago.
November 13, 2016

***

## Objective

* **Part I: Introduction to RNAseq technology, clinical application and analysis pipeline**
    * Learn the background and clinical application of RNAseq
    * Learn the best-practice analysis protocol
* **Part II: Practice how to perform downstream analysis of RNAseq data (hands-on)**
    * Detect differentially expressed genes (DEGs) between conditions
    * Identify pathways / network enriched in genes of interest
    * Generate high-quality publication figures (Principle Component Analysis (PCA), heatmap, pathway, etc.)
* **Part III: Practice how to associate RNAseq with clincial data (hands-on)**
    * The Cancer Genome Atlas (TCGA) 
    * NCI Genomics Data Commons (GDC) hosts the new harmonized TCGA data

***

## Dataset

* **Data to run RNAseq analysis**
    * Two groups (*PRDM11* KO vs WT, U2932 cells), 6 samples
    * Aim to identify DEGs / pathways altered between KO and WT groups
    * Fog. et al. 2015. [Loss of PRDM11 promotes MYC-driven lymphomagenesis](http://www.bloodjournal.org/content/125/8/1272.long?sso-checked=true). Blood 125(8):1272-81

* **Data to associate RNAseq with clinical outcome**
    * TCGA ovarian cancer, 379 primary tumors
    * Aim to use gene expression to group patients into subtypes and detect survival differences
    * The Cancer Genome Atlas Research Network. 2011. [Integrated genomic analyses of ovarian carcinoma](http://www.nature.com/nature/journal/v474/n7353/full/nature10166.html). Nature 474, 609–615

***

## Workflow

<img src='notebook_ext/ipynb_data/assets/RNAseq.workflow.v2.png', title = 'RNAseq workflow', width = 1000, height = 1000>

***

## Part I: Introduction to RNAseq technology, clinical application and analysis pipeline

### 1.1 What is RNAseq

### 1.2 Clinical applications 

### 1.3 How to perform RNAseq analysis

* From raw sequencing to transcript quantification (steps 1-5) are **automated through BigDataScript pipeline**.
* DEG and pathway analysis (steps 6 to 7) can be automated, but it is recommended to **perform the analysis interactively to better interpret the results**.

*Due to time limit, we will not run the pipeline in this workshop. Since it is automated, participants are encouraged to practice it post-workshop following the instructions at [Github](https://github.com/riyuebao/CRI-Workshop-Nov2016-RNAseq)*

### 1.3.1 Quality accessment of raw sequencing files

## Part II: Practice the downstream analysis of RNAseq data

## Part III: Practice how to associate RNAseq with clincial data 